# Exploratory study of the REF 2021 submission data

In [1]:
import os
import sys
import pandas as pd

sys.path.append('../src/')
import codebook as cb
import read_write as rw
import visualisations as vis

## Outputs

In [2]:
fname = rw.DATA_PPROC_OUTPUTS
dset = pd.read_csv(os.path.join(rw.PROJECT_PATH, fname),
                   index_col=None,
                   dtype={0: str})
print(f"Read {fname}: {dset.shape[0]} records")

Read data/processed/Outputs_ppreprocessed.csv.gz: 185354 records
